In [1]:
import glob
import os
import numpy as np
import pandas as pd
import tensorflow as tf 
from statistics import mode

In [2]:
#incomplete prepare data method for us to check features
def prepare_data(meta_filename, filepath, \
    use_composer = True, use_composition = True, use_movement = True, use_ensemble = True):
    """
    main method to prepare data.
    params:
    meta_filename is the file name of the meta files
    filepath is the path of the folder that contains all the data
    use_* indicates whether to use those meta features as the data feature 
    """
    def load_music_data(filepath):
        all_filenames = glob.glob(filepath + "/*.csv")
        music_data = []
        for filename in all_filenames:
            temp = pd.read_csv(filename)
            temp['id'] = os.path.basename(filename[:-4])
            music_data.append(temp)
        music_data = pd.concat(music_data, axis=0, ignore_index=True)
        music_data['id'] = music_data['id'].astype(str).astype(int)

        return music_data
        
    #load meta-data
    meta_data = pd.read_csv(meta_filename)

    #load music
    music_data = load_music_data(filepath)

    #merge two data
    all_data = meta_data.merge(music_data, on = 'id')
    all_data = all_data.drop(['source', 'transcriber', 'catalog_name'], axis=1)

    return all_data

In [3]:
data = prepare_data('/Users/zyl/Desktop/CS1470/FinalProject/data/musicnet_metadata.csv', 
'/Users/zyl/Desktop/CS1470/FinalProject/data/musicnet/test_labels')

composition = np.unique(data['composition'].to_numpy())
print(f'There are {len(composition)} compositions included:')
print(composition)

There are 10 compositions included:
['Cello Suite 4' 'Piano Sonata No 30 in E major' 'Piano Sonata in C minor'
 'Quartet No 53 in D major' 'Serenade in E-flat major'
 'Sextet in E-flat major for Winds' 'String Quartet No 13 in B-flat major'
 'Violin Partita No 3 in E major' 'Violin Sonata No 10 in G major'
 'WTK I, No. 5: Prelude and Fugue in D major']


In [4]:
print(set([c.split()[-2] for c in composition]))

{'D', 'E', 'for', 'Suite', 'C', 'E-flat', 'B-flat', 'G'}


In [5]:
notes = data['note'].to_numpy()
print(min(notes))
print(max(notes))

28
95


In [6]:
def calculate_degree(notes, keys):
    print("all unique keys: ", np.unique(keys))
    print("shape of notes array: ", np.array(notes).shape)
    re = []
    for (n, k) in zip(notes, keys):
        diff = (n % 12) - k
        diff = diff + 12 if diff < 0 else diff
        re.append(diff)
    
    return np.array(re)

def find_keys(all_data):
    ref = {'C':0, 'C-sharp':1, 'D-flat':1, 'D':2, 'D-sharp':3, 'E-flat':3, 'E':4, \
        'F':5, 'F-sharp':6, 'G-flat':6, 'G':7, 'G-sharp':8, 'A-flat':8, 'A':9, 
        'A-sharp':10, 'B-flat':10, 'B':11}

    data = np.array(all_data['composition'])
    unresolved = []

    for i, d in enumerate(data):
        if d.split()[-2] in ref:
            data[i] = ref[d.split()[-2]]
        else:
            if d not in unresolved:
                unresolved.append(d)
    
    unresolved_dict = {}
    for composition in unresolved:
        notes_in_piece = np.array(all_data[all_data['composition'] == composition]['note'])
        notes_in_piece = [i % 12 for i in notes_in_piece]
        #we assume the key is the most common note in the piece
        unresolved_dict[composition] = mode(notes_in_piece)

    for i, d in enumerate(data):
        if d in unresolved_dict.keys():
            data[i] = unresolved_dict[d]

    return np.array(data)

In [7]:
#see if check note degree works
data = prepare_data('/Users/zyl/Desktop/CS1470/FinalProject/data/musicnet_metadata.csv', 
'/Users/zyl/Desktop/CS1470/FinalProject/data/musicnet/test_labels')
data['note_degree'] = calculate_degree(data['note'].to_numpy(), find_keys(data))
data.head()

all unique keys:  [0 2 3 4 7 10]
shape of notes array:  (13589,)


,id,composer,composition,movement,ensemble,seconds,start_time,end_time,instrument,note,start_beat,end_beat,note_value,note_degree
0,1759,Schubert,Piano Sonata in C minor,3. Menuetto and Trio,Solo Piano,194,90078,124382,1,63,0.0,1.000,Quarter,3
1,1759,Schubert,Piano Sonata in C minor,3. Menuetto and Trio,Solo Piano,194,90078,124382,1,75,0.0,1.000,Quarter,3
2,1759,Schubert,Piano Sonata in C minor,3. Menuetto and Trio,Solo Piano,194,90078,110558,1,48,0.0,0.375,Dotted Sixteenth,0
3,1759,Schubert,Piano Sonata in C minor,3. Menuetto and Trio,Solo Piano,194,114654,122334,1,55,0.5,0.375,Dotted Sixteenth,7
4,1759,Schubert,Piano Sonata in C minor,3. Menuetto and Trio,Solo Piano,194,124382,139742,1,65,1.0,1.000,Quarter,5


In [8]:
print(np.unique(data['movement']))

['1. Prelude' '2. Prestissimo' '2. Presto' '3. Menuetto and Trio'
 '3. Menuetto. Quasi Allegretto - Trio' '3. Menuetto: Allegretto'
 '3. Scherzo: Allegro - Trio' '4. Menuetto' '6. Bourree' '6. Gigue']


In [9]:
def one_hot_encoding(array):
    #print(np.unique(array))
    value_dict = {value : index for index, value in enumerate(np.unique(array))}
    #print(value_dict)
    array_trans = [value_dict[value] for value in array]
    ohe = tf.one_hot(array_trans, len(value_dict))
    return list(ohe)

In [10]:
data['movement_ohe'] = one_hot_encoding(data['movement'])
print(data['movement_ohe'][0])
print(data['movement_ohe'][5000])

tf.Tensor([0. 0. 0. 1. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 1. 0. 0. 0. 0.], shape=(10,), dtype=float32)


In [11]:
print(np.unique(data['composer']))

['Bach' 'Beethoven' 'Haydn' 'Mozart' 'Schubert']


In [12]:
data['composer_ohe'] = one_hot_encoding(data['composer'].to_numpy())
print(data['composer_ohe'][0])
print(data['composer_ohe'][5000])

tf.Tensor([0. 0. 0. 0. 1.], shape=(5,), dtype=float32)
tf.Tensor([0. 0. 1. 0. 0.], shape=(5,), dtype=float32)


In [13]:
print(np.unique(data['ensemble']))

['Accompanied Violin' 'Pairs Clarinet-Horn-Bassoon' 'Solo Cello'
 'Solo Piano' 'Solo Violin' 'String Quartet']


In [14]:
data['ensemble_ohe'] = one_hot_encoding(data['ensemble'])
print(data['ensemble_ohe'][0])
print(data['ensemble_ohe'][5000])

tf.Tensor([0. 0. 0. 1. 0. 0.], shape=(6,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 1.], shape=(6,), dtype=float32)


In [15]:
data.head()

,id,composer,composition,movement,ensemble,seconds,start_time,end_time,instrument,note,start_beat,end_beat,note_value,note_degree,movement_ohe,composer_ohe,ensemble_ohe
0,1759,Schubert,Piano Sonata in C minor,3. Menuetto and Trio,Solo Piano,194,90078,124382,1,63,0.0,1.000,Quarter,3,"(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T..."
1,1759,Schubert,Piano Sonata in C minor,3. Menuetto and Trio,Solo Piano,194,90078,124382,1,75,0.0,1.000,Quarter,3,"(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T..."
2,1759,Schubert,Piano Sonata in C minor,3. Menuetto and Trio,Solo Piano,194,90078,110558,1,48,0.0,0.375,Dotted Sixteenth,0,"(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T..."
3,1759,Schubert,Piano Sonata in C minor,3. Menuetto and Trio,Solo Piano,194,114654,122334,1,55,0.5,0.375,Dotted Sixteenth,7,"(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T..."
4,1759,Schubert,Piano Sonata in C minor,3. Menuetto and Trio,Solo Piano,194,124382,139742,1,65,1.0,1.000,Quarter,5,"(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T..."


In [1]:
#final test
import preprocessing
data_class = preprocessing.DataClass('/Users/zyl/Desktop/CS1470/FinalProject/data/musicnet_metadata.csv', 
'/Users/zyl/Desktop/CS1470/FinalProject/data/musicnet/test_labels')
data_class.data.head()

,id,composition,movement,ensemble,seconds,start_time,end_time,instrument,note,start_beat,end_beat,note_value,key,note_degree
0,1759,Piano Sonata in C minor,"(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",194,90078,124382,1,63,0.0,1.000,"(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",0,3
1,1759,Piano Sonata in C minor,"(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",194,90078,124382,1,75,0.0,1.000,"(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",0,3
2,1759,Piano Sonata in C minor,"(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",194,90078,110558,1,48,0.0,0.375,"(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",0,0
3,1759,Piano Sonata in C minor,"(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",194,114654,122334,1,55,0.5,0.375,"(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",0,7
4,1759,Piano Sonata in C minor,"(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",194,124382,139742,1,65,1.0,1.000,"(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",0,5


In [2]:
data_class.data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13589 entries, 0 to 13588
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           13589 non-null  int64  
 1   composition  13589 non-null  object 
 2   movement     13589 non-null  object 
 3   ensemble     13589 non-null  object 
 4   seconds      13589 non-null  int64  
 5   start_time   13589 non-null  int64  
 6   end_time     13589 non-null  int64  
 7   instrument   13589 non-null  int64  
 8   note         13589 non-null  int64  
 9   start_beat   13589 non-null  float64
 10  end_beat     13589 non-null  float64
 11  note_value   13589 non-null  object 
 12  key          13589 non-null  object 
 13  note_degree  13589 non-null  int64  
dtypes: float64(2), int64(7), object(5)
memory usage: 2.1+ MB


In [3]:
X = data_class.encode_nn_ready()
X.shape

(13589, 38)

In [4]:
example = X[0]
example

array([1.94000e+02, 9.00780e+04, 1.24382e+05, 1.00000e+00, 6.30000e+01,
       0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
       0.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00,
       0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
       0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00,
       0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
       0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00,
       0.00000e+00, 0.00000e+00, 3.00000e+00], dtype=float32)

In [5]:
decoded = data_class.decode_to_datafrom([example])
decoded

,seconds,start_time,end_time,instrument,note,start_beat,end_beat,movement,ensemble
0,194,90078,124382,1,63,0.0,1.0,3. Scherzo: Allegro - Trio,Accompanied Violin
